# Build the "HomeMatch" application

## Step 1: Setting Up the Python Application

In [142]:
import os
import re
import csv
import lancedb
import pandas as pd
import numpy as np
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from lancedb.pydantic import vector, LanceModel

openai_api_key = os.getenv("OPENAI_API_KEY")

## Step2: Generating Real Estate Listings

In [190]:
listings = [
    {
        "Neighborhood": "Green Oaks",
        "Price": "$800,000",
        "Bedrooms": 3,
        "Bathrooms": 2,
        "House Size": "2,000 sqft"
    }
]

prompt_template = """
Please generate 10 real estate listings for a property with the following details:
Neighborhood:
Price:
Bedrooms:
Bathrooms:
House Size:
Description:
Neighborhood Description:
"""

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
    

prompt = PromptTemplate(
    template=prompt_template
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)


response = chain.run(listings[0])
print(response)

pattern = re.compile(
    r"Neighborhood: (.*?)\n"
    r"Price: (.*?)\n"
    r"Bedrooms: (\d+)\n"
    r"Bathrooms: ([\d.]+)\n"
    r"House Size: (.*?)\n"
    r"Description: (.*?)\n"
    r"Neighborhood Description: (.*?)(?=\n\d+\.|$)",
    re.DOTALL
)

matches = pattern.findall(response)

listings = []
for match in matches:
    listing = {
        "neighborhood": match[0].strip(),
        "price": match[1].strip(),
        "bedrooms": int(match[2].strip()),
        "bathrooms": float(match[3].strip()),
        "house_size": match[4].strip(),
        "description": match[5].strip(),
        "neighborhood_description": match[6].strip()
    }
    listings.append(listing)

for listing in listings:
    print(listing)

# Save to CSV
csv_file = "listings.csv"
csv_columns = ["house_id", "neighborhood", "price", "bedrooms", "bathrooms", "house_size", "description", "neighborhood_description", "full_description"]

try:
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=csv_columns)
        writer.writeheader()
        for i,listing in enumerate(listings):
            listing["house_id"] = i + 1
            listing["full_description"] = f"price: {listing["price"]}, bedrooms: {listing["bedrooms"]}, bathrooms: {listing["bathrooms"]}, house_size: {listing["house_size"]}, full_description{listing["description"] + listing["neighborhood_description"]}"
            writer.writerow(listing)
    print(f"Listings have been saved to {csv_file}")
except IOError:
    print("I/O error")

1. Neighborhood: Upper East Side, Manhattan, NY
Price: $2,500,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Stunning luxury apartment with high-end finishes and breathtaking views of Central Park.
Neighborhood Description: Upscale neighborhood known for its exclusive shops, fine dining, and proximity to world-class museums.

2. Neighborhood: Venice Beach, Los Angeles, CA
Price: $1,200,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,500 sqft
Description: Charming beach bungalow with a private backyard and just steps away from the ocean.
Neighborhood Description: Bohemian beach community with a vibrant arts scene, trendy shops, and eclectic dining options.

3. Neighborhood: Buckhead, Atlanta, GA
Price: $800,000
Bedrooms: 4
Bathrooms: 3
House Size: 3,500 sqft
Description: Spacious family home with a gourmet kitchen, large backyard, and a bonus room.
Neighborhood Description: Upscale residential area known for its luxury shopping, fine dining, and prestigious schools.

4. Nei

## Step3: Storing Listings in a Vector Database

In [191]:
df = pd.read_csv("./listings.csv", header=0)
df = df.set_index("house_id").reindex(df.house_id)
df

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description
house_id,,,,,,,,
1,"Upper East Side, Manhattan, NY","$2,500,000",3,2.0,"2,000 sqft",Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: $2,500,000, bedrooms: 3, bathrooms: 2.0..."
2,"Venice Beach, Los Angeles, CA","$1,200,000",2,1.0,"1,500 sqft",Charming beach bungalow with a private backyar...,Bohemian beach community with a vibrant arts s...,"price: $1,200,000, bedrooms: 2, bathrooms: 1.0..."
3,"Buckhead, Atlanta, GA","$800,000",4,3.0,"3,500 sqft","Spacious family home with a gourmet kitchen, l...",Upscale residential area known for its luxury ...,"price: $800,000, bedrooms: 4, bathrooms: 3.0, ..."
4,"South Beach, Miami, FL","$1,500,000",1,1.5,"1,200 sqft","Modern condo with ocean views, high ceilings, ...",Trendy neighborhood known for its vibrant nigh...,"price: $1,500,000, bedrooms: 1, bathrooms: 1.5..."
5,"Pacific Heights, San Francisco, CA","$3,000,000",5,4.0,"4,000 sqft",Elegant Victorian home with a landscaped garde...,Prestigious residential area with historic man...,"price: $3,000,000, bedrooms: 5, bathrooms: 4.0..."
6,"Georgetown, Washington, D.C.","$1,800,000",3,2.5,"2,500 sqft","Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: $1,800,000, bedrooms: 3, bathrooms: 2.5..."
7,"Old Town, Alexandria, VA","$900,000",2,2.0,"1,800 sqft",Quaint colonial-style home with a fenced backy...,Charming waterfront neighborhood with historic...,"price: $900,000, bedrooms: 2, bathrooms: 2.0, ..."
8,"Cherry Creek, Denver, CO","$1,100,000",4,3.0,"2,800 sqft","Modern townhouse with a rooftop deck, open flo...",Upscale shopping and dining district with art ...,"price: $1,100,000, bedrooms: 4, bathrooms: 3.0..."
9,"The Woodlands, Houston, TX","$700,000",3,2.5,"2,200 sqft","Ranch-style home with a pool, updated kitchen,...","Master-planned community with parks, golf cour...","price: $700,000, bedrooms: 3, bathrooms: 2.5, ..."


In [192]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
documents = [Document(content) for content in df['full_description']]
documents

[Document(page_content='price: $2,500,000, bedrooms: 3, bathrooms: 2.0, house_size: 2,000 sqft, full_descriptionStunning luxury apartment with high-end finishes and breathtaking views of Central Park.Upscale neighborhood known for its exclusive shops, fine dining, and proximity to world-class museums.'),
 Document(page_content='price: $1,200,000, bedrooms: 2, bathrooms: 1.0, house_size: 1,500 sqft, full_descriptionCharming beach bungalow with a private backyard and just steps away from the ocean.Bohemian beach community with a vibrant arts scene, trendy shops, and eclectic dining options.'),
 Document(page_content='price: $800,000, bedrooms: 4, bathrooms: 3.0, house_size: 3,500 sqft, full_descriptionSpacious family home with a gourmet kitchen, large backyard, and a bonus room.Upscale residential area known for its luxury shopping, fine dining, and prestigious schools.'),
 Document(page_content='price: $1,500,000, bedrooms: 1, bathrooms: 1.5, house_size: 1,200 sqft, full_descriptionMode

In [193]:
embeddings = embeddings_model.embed_documents([doc.page_content for doc in documents])
len(embeddings)

10

In [194]:
df["vector"] = embeddings
df

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description,vector
house_id,,,,,,,,,
1,"Upper East Side, Manhattan, NY","$2,500,000",3,2.0,"2,000 sqft",Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: $2,500,000, bedrooms: 3, bathrooms: 2.0...","[0.0009346878538940622, 0.022937831054276936, ..."
2,"Venice Beach, Los Angeles, CA","$1,200,000",2,1.0,"1,500 sqft",Charming beach bungalow with a private backyar...,Bohemian beach community with a vibrant arts s...,"price: $1,200,000, bedrooms: 2, bathrooms: 1.0...","[0.0005224690988281046, 0.0182258184065481, 0...."
3,"Buckhead, Atlanta, GA","$800,000",4,3.0,"3,500 sqft","Spacious family home with a gourmet kitchen, l...",Upscale residential area known for its luxury ...,"price: $800,000, bedrooms: 4, bathrooms: 3.0, ...","[0.009080679573492808, 0.029692586014245216, -..."
4,"South Beach, Miami, FL","$1,500,000",1,1.5,"1,200 sqft","Modern condo with ocean views, high ceilings, ...",Trendy neighborhood known for its vibrant nigh...,"price: $1,500,000, bedrooms: 1, bathrooms: 1.5...","[0.004585443990518324, 0.01543820181914627, 0...."
5,"Pacific Heights, San Francisco, CA","$3,000,000",5,4.0,"4,000 sqft",Elegant Victorian home with a landscaped garde...,Prestigious residential area with historic man...,"price: $3,000,000, bedrooms: 5, bathrooms: 4.0...","[0.009108896113661929, 0.002429804712643047, -..."
6,"Georgetown, Washington, D.C.","$1,800,000",3,2.5,"2,500 sqft","Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: $1,800,000, bedrooms: 3, bathrooms: 2.5...","[-0.0024082558845829496, 0.02054696396418098, ..."
7,"Old Town, Alexandria, VA","$900,000",2,2.0,"1,800 sqft",Quaint colonial-style home with a fenced backy...,Charming waterfront neighborhood with historic...,"price: $900,000, bedrooms: 2, bathrooms: 2.0, ...","[0.002404176244672894, 0.01805585508199624, 0...."
8,"Cherry Creek, Denver, CO","$1,100,000",4,3.0,"2,800 sqft","Modern townhouse with a rooftop deck, open flo...",Upscale shopping and dining district with art ...,"price: $1,100,000, bedrooms: 4, bathrooms: 3.0...","[0.0019086132562708379, 0.019949128891193876, ..."
9,"The Woodlands, Houston, TX","$700,000",3,2.5,"2,200 sqft","Ranch-style home with a pool, updated kitchen,...","Master-planned community with parks, golf cour...","price: $700,000, bedrooms: 3, bathrooms: 2.5, ...","[-0.0031795084426606483, 0.021725574156050732,..."


In [195]:
class RealEstate(LanceModel):
    neighborhood: str
    vector: vector(len(embeddings[0]))
    price: str
    bedrooms: int
    bathrooms: float
    house_size: str
    description: str
    neighborhood_description: str

db = lancedb.connect("~/.lancedb")
table_name = "real_estate"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, data=df)

In [196]:
table.count_rows()

10

In [197]:
table.to_pandas()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description,vector
0,"Upper East Side, Manhattan, NY","$2,500,000",3,2.0,"2,000 sqft",Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: $2,500,000, bedrooms: 3, bathrooms: 2.0...","[0.00093468785, 0.02293783, 0.0069465935, -0.0..."
1,"Venice Beach, Los Angeles, CA","$1,200,000",2,1.0,"1,500 sqft",Charming beach bungalow with a private backyar...,Bohemian beach community with a vibrant arts s...,"price: $1,200,000, bedrooms: 2, bathrooms: 1.0...","[0.0005224691, 0.018225819, 0.009578054, 0.008..."
2,"Buckhead, Atlanta, GA","$800,000",4,3.0,"3,500 sqft","Spacious family home with a gourmet kitchen, l...",Upscale residential area known for its luxury ...,"price: $800,000, bedrooms: 4, bathrooms: 3.0, ...","[0.009080679, 0.029692587, -0.013988276, -0.01..."
3,"South Beach, Miami, FL","$1,500,000",1,1.5,"1,200 sqft","Modern condo with ocean views, high ceilings, ...",Trendy neighborhood known for its vibrant nigh...,"price: $1,500,000, bedrooms: 1, bathrooms: 1.5...","[0.004585444, 0.015438202, 0.0024790564, -0.01..."
4,"Pacific Heights, San Francisco, CA","$3,000,000",5,4.0,"4,000 sqft",Elegant Victorian home with a landscaped garde...,Prestigious residential area with historic man...,"price: $3,000,000, bedrooms: 5, bathrooms: 4.0...","[0.009108896, 0.0024298048, -0.004921954, -0.0..."
5,"Georgetown, Washington, D.C.","$1,800,000",3,2.5,"2,500 sqft","Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: $1,800,000, bedrooms: 3, bathrooms: 2.5...","[-0.0024082558, 0.020546963, 0.0068881996, -0...."
6,"Old Town, Alexandria, VA","$900,000",2,2.0,"1,800 sqft",Quaint colonial-style home with a fenced backy...,Charming waterfront neighborhood with historic...,"price: $900,000, bedrooms: 2, bathrooms: 2.0, ...","[0.0024041762, 0.018055854, 0.00035919537, 0.0..."
7,"Cherry Creek, Denver, CO","$1,100,000",4,3.0,"2,800 sqft","Modern townhouse with a rooftop deck, open flo...",Upscale shopping and dining district with art ...,"price: $1,100,000, bedrooms: 4, bathrooms: 3.0...","[0.0019086133, 0.019949129, -0.0037234926, -0...."
8,"The Woodlands, Houston, TX","$700,000",3,2.5,"2,200 sqft","Ranch-style home with a pool, updated kitchen,...","Master-planned community with parks, golf cour...","price: $700,000, bedrooms: 3, bathrooms: 2.5, ...","[-0.0031795085, 0.021725575, -0.0132554155, -0..."
9,"North End, Boston, MA","$1,600,000",2,2.0,"1,500 sqft","Luxury penthouse with a private terrace, gourm...",Historic neighborhood known for its Italian re...,"price: $1,600,000, bedrooms: 2, bathrooms: 2.0...","[0.014032029, 0.012445856, 0.007891371, -0.014..."


In [198]:
query = "nice house with a view on the sea"
query_vector = embeddings_model.embed_documents([query])
results = table.search(query_vector[0]).limit(2).to_pydantic(RealEstate)
results

[RealEstate(neighborhood='South Beach, Miami, FL', vector=FixedSizeList(dim=1536), price='$1,500,000', bedrooms=1, bathrooms=1.5, house_size='1,200 sqft', description='Modern condo with ocean views, high ceilings, and access to a rooftop pool.', neighborhood_description='Trendy neighborhood known for its vibrant nightlife, white sand beaches, and Art Deco architecture.'),
 RealEstate(neighborhood='Venice Beach, Los Angeles, CA', vector=FixedSizeList(dim=1536), price='$1,200,000', bedrooms=2, bathrooms=1.0, house_size='1,500 sqft', description='Charming beach bungalow with a private backyard and just steps away from the ocean.', neighborhood_description='Bohemian beach community with a vibrant arts scene, trendy shops, and eclectic dining options.')]

## Step4: Building the User Preference Interface

In [203]:
questions = [
    "How big do you want your house to be?",
    "What are the 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"
]


answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

def format_qa(questions, answers):
    formatted_qa = ""
    for q, a in zip(questions, answers):
        formatted_qa += f"Q: {q}\nA: {a}\n"
    return formatted_qa

formatted_qa = format_qa(questions, answers)

prompt_template = """
{formatted_qa}

Please generate the following details based on Q&As:
Neighborhood:
Price:
Bedrooms:
Bathrooms:
House Size:
Description:
Neighborhood Description:
"""

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    openai_api_key=openai_api_key
)
    

prompt = PromptTemplate(
    template=prompt_template
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)


response = chain.run(formatted_qa)

pattern = re.compile(
    r"Neighborhood: (.*?)\n"
    r"Price: (.*?)\n"
    r"Bedrooms: (\d+)\n"
    r"Bathrooms: ([\d.]+)\n"
    r"House Size: (.*?)\n"
    r"Description: (.*?)\n"
    r"Neighborhood Description: (.*?)(?=\n\d+\.|$)",
    re.DOTALL
)

matches = pattern.findall(response)

preferences = []
for match in matches:
    preference = {
        "neighborhood": match[0].strip(),
        "price": match[1].strip(),
        "bedrooms": int(match[2].strip()),
        "bathrooms": float(match[3].strip()),
        "house_size": match[4].strip(),
        "description": match[5].strip(),
        "neighborhood_description": match[6].strip()
    }
    preferences.append(preference)

preferences[0]

{'neighborhood': 'Suburban neighborhood with access to urban amenities',
 'price': 'TBD based on market value',
 'bedrooms': 3,
 'bathrooms': 2.0,
 'house_size': 'TBD',
 'description': 'A comfortable three-bedroom house with a spacious kitchen, cozy living room, backyard for gardening, two-car garage, and modern, energy-efficient heating system.',
 'neighborhood_description': 'A quiet neighborhood with good local schools, convenient shopping options, easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. Balances suburban tranquility with access to urban amenities like restaurants and theaters.'}

## Step5: Searching Based on Preferences

In [205]:
def get_full_description(pref):
        return f"price: {pref["price"]}, bedrooms: {pref["bedrooms"]}, bathrooms: {pref["bathrooms"]}, house_size: {pref["house_size"]}, full_description{pref["description"] + pref["neighborhood_description"]}"

query = get_full_description(preferences[0])
query_vector = embeddings_model.embed_documents([query])
results = table.search(query_vector[0]).limit(2).to_pydantic(RealEstate)
results

[RealEstate(neighborhood='Buckhead, Atlanta, GA', vector=FixedSizeList(dim=1536), price='$800,000', bedrooms=4, bathrooms=3.0, house_size='3,500 sqft', description='Spacious family home with a gourmet kitchen, large backyard, and a bonus room.', neighborhood_description='Upscale residential area known for its luxury shopping, fine dining, and prestigious schools.'),
 RealEstate(neighborhood='Georgetown, Washington, D.C.', vector=FixedSizeList(dim=1536), price='$1,800,000', bedrooms=3, bathrooms=2.5, house_size='2,500 sqft', description='Renovated townhouse with a private patio, gourmet kitchen, and a fireplace.', neighborhood_description='Historic neighborhood known for its cobblestone streets, upscale shops, and waterfront dining options.')]

## Step 6: Personalizing Listing Descriptions

In [237]:
def augment_customer_preference(customer_input):
    prompt_template = """
    {customer_input}
    
    Please generate the following details based on the customer_input. Please enter the price, bedrooms, bathrooms, and house size as a number.:
    Neighborhood:
    Price:
    Bedrooms:
    Bathrooms:
    House Size:
    Description:
    Neighborhood Description:
    """
    
    model_name = "gpt-3.5-turbo"
    temperature = 0.0
    openai_api_key = os.getenv("OPENAI_API_KEY")
    
    if not openai_api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set")

    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        openai_api_key=openai_api_key
    )
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["customer_input"]
    )
    
    chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    
    response = chain.run({"customer_input": customer_input})
    print("Generated Response:\n", response)
    
    pattern = re.compile(
        r"Neighborhood: (.*?)\n"
        r"Price: (.*?)\n"
        r"Bedrooms: (\d+)\+?\n"
        r"Bathrooms: ([\d.]+)\+?\n"
        r"House Size: (.*?)\n"
        r"Description: (.*?)\n"
        r"Neighborhood Description: (.*?)(?=\n|$)",
        re.DOTALL
    )
    
    matches = pattern.findall(response)

    preferences = []
    for match in matches:
        preference = {
            "neighborhood": match[0].strip(),
            "price": match[1].strip(),
            "bedrooms": int(match[2].strip()),
            "bathrooms": float(match[3].strip()),
            "house_size": match[4].strip(),
            "description": match[5].strip(),
            "neighborhood_description": match[6].strip()
        }
        preferences.append(preference)

    return preferences[0] if preferences else None

customer_input = """
I am looking for a comfortable four-bedroom house that can accommodate my family of five.
It should be in a safe neighborhood with good local schools. A large backyard is essential for my children to play in.
Additional amenities we need include a swimming pool, a two-car garage, and a home office.
The house should be in a suburban area with parks and recreational facilities nearby.
Easy access to public transportation and major highways is also important.
"""
pref = augment_customer_preference(customer_input)
pref

Generated Response:
 Neighborhood: Suburban area with parks and recreational facilities nearby
Price: $600,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sq ft
Description: Comfortable four-bedroom house with a large backyard, swimming pool, two-car garage, and home office.
Neighborhood Description: Safe neighborhood with good local schools, easy access to public transportation and major highways.


{'neighborhood': 'Suburban area with parks and recreational facilities nearby',
 'price': '$600,000',
 'bedrooms': 4,
 'bathrooms': 3.0,
 'house_size': '2,500 sq ft',
 'description': 'Comfortable four-bedroom house with a large backyard, swimming pool, two-car garage, and home office.',
 'neighborhood_description': 'Safe neighborhood with good local schools, easy access to public transportation and major highways.'}

In [238]:
def find_estate(pref):
    query = get_full_description(pref)
    query_vector = embeddings_model.embed_documents([query])
    results = table.search(query_vector[0]).limit(2).to_pydantic(RealEstate)
    return results

search_results = find_estate(pref)
search_results

[RealEstate(neighborhood='Buckhead, Atlanta, GA', vector=FixedSizeList(dim=1536), price='$800,000', bedrooms=4, bathrooms=3.0, house_size='3,500 sqft', description='Spacious family home with a gourmet kitchen, large backyard, and a bonus room.', neighborhood_description='Upscale residential area known for its luxury shopping, fine dining, and prestigious schools.'),
 RealEstate(neighborhood='The Woodlands, Houston, TX', vector=FixedSizeList(dim=1536), price='$700,000', bedrooms=3, bathrooms=2.5, house_size='2,200 sqft', description='Ranch-style home with a pool, updated kitchen, and a spacious backyard.', neighborhood_description='Master-planned community with parks, golf courses, shopping centers, and top-rated schools.')]

## Test

In [239]:
customer_input = """
I want a modern two-bedroom apartment in the city.
The property should be close to my workplace, with vibrant nightlife and many restaurants and cafes nearby.
Essential amenities include a gym, a rooftop terrace, and high-speed internet.
The neighborhood should have easy access to metro stations and be bike-friendly.
I prefer a highly urban area with lots of activities and events.
"""

pref = augment_customer_preference(customer_input)
search_results = find_estate(pref)
search_results

Generated Response:
 Neighborhood: Downtown City Center
Price: $2500
Bedrooms: 2
Bathrooms: 2
House Size: 1000 sq ft
Description: Modern two-bedroom apartment in the heart of the city with a gym, rooftop terrace, and high-speed internet.
Neighborhood Description: Vibrant area with plenty of nightlife, restaurants, cafes, easy access to metro stations, and bike-friendly streets. Perfect for those who enjoy urban living and being close to various activities and events.


[RealEstate(neighborhood='Cherry Creek, Denver, CO', vector=FixedSizeList(dim=1536), price='$1,100,000', bedrooms=4, bathrooms=3.0, house_size='2,800 sqft', description='Modern townhouse with a rooftop deck, open floor plan, and a two-car garage.', neighborhood_description='Upscale shopping and dining district with art galleries, boutiques, and outdoor cafes.'),
 RealEstate(neighborhood='Upper East Side, Manhattan, NY', vector=FixedSizeList(dim=1536), price='$2,500,000', bedrooms=3, bathrooms=2.0, house_size='2,000 sqft', description='Stunning luxury apartment with high-end finishes and breathtaking views of Central Park.', neighborhood_description='Upscale neighborhood known for its exclusive shops, fine dining, and proximity to world-class museums.')]

In [240]:
customer_input = """
I desire a three-bedroom beachfront house with direct access to the beach and stunning ocean views.
The property should be close to facilities for water sports.
It should also include a private dock, a swimming pool, and a spacious deck.
I need easy access to a marina and coastal highways.
The ideal neighborhood is a coastal town with a relaxed atmosphere and good tourist facilities.
"""

pref = augment_customer_preference(customer_input)
print(pref)
search_results = find_estate(pref)
search_results

Generated Response:
 Neighborhood: Coastal town with a relaxed atmosphere and good tourist facilities
Price: $1,500,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,500 sq ft
Description: Three-bedroom beachfront house with direct access to the beach, stunning ocean views, private dock, swimming pool, spacious deck, and close proximity to water sports facilities.
Neighborhood Description: The coastal town offers a relaxed atmosphere with good tourist facilities, easy access to a marina, and coastal highways.
{'neighborhood': 'Coastal town with a relaxed atmosphere and good tourist facilities', 'price': '$1,500,000', 'bedrooms': 3, 'bathrooms': 2.0, 'house_size': '2,500 sq ft', 'description': 'Three-bedroom beachfront house with direct access to the beach, stunning ocean views, private dock, swimming pool, spacious deck, and close proximity to water sports facilities.', 'neighborhood_description': 'The coastal town offers a relaxed atmosphere with good tourist facilities, easy access to a mar

[RealEstate(neighborhood='Venice Beach, Los Angeles, CA', vector=FixedSizeList(dim=1536), price='$1,200,000', bedrooms=2, bathrooms=1.0, house_size='1,500 sqft', description='Charming beach bungalow with a private backyard and just steps away from the ocean.', neighborhood_description='Bohemian beach community with a vibrant arts scene, trendy shops, and eclectic dining options.'),
 RealEstate(neighborhood='South Beach, Miami, FL', vector=FixedSizeList(dim=1536), price='$1,500,000', bedrooms=1, bathrooms=1.5, house_size='1,200 sqft', description='Modern condo with ocean views, high ceilings, and access to a rooftop pool.', neighborhood_description='Trendy neighborhood known for its vibrant nightlife, white sand beaches, and Art Deco architecture.')]